In [1]:
import os
import shutil
import glob
import json
import random
import scipy.io as sio
import pandas as pd
import numpy as np
import re
import nibabel as nib
from nilearn.plotting import plot_stat_map
import pickle

/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-nvr-fmri/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
os.chdir('/Users/zeynepenkavi/Documents/RangelLab/NovelVsRepeated/fmri/analysis/01_level1/cluster_scripts')

In [3]:
from level1_utils import run_level1

In [4]:
data_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri'

In [ ]:
subnum = '601'
session = '02'
task = 'binaryChoice'
mnum = 'model1'
space = 'MNI152NLin2009cAsym_res-2'
out_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri/derivatives/nilearn/glm/level1/%s/%s'%(task, mnum)

In [ ]:
# run_level1(subnum, session, task, mnum, data_path, out_path, space)

In [5]:
subnum = '601'
session = '02'
task = 'yesNo'
mnum = 'model1'
space = 'MNI152NLin2009cAsym_res-2'

data_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri'
out_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri/derivatives/nilearn/glm/level1/%s/%s'%(task, mnum)

In [6]:
run_level1(subnum, session, task, mnum, data_path, out_path, space)

A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-601 ses-02 task-yesNo run-01
***********************************************


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-nvr-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-601 ses-02 task-yesNo run-02
***********************************************
***********************************************
Running fixed effects GLM for all runs of sub-601 ses-02 task-yesNo
***********************************************


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-nvr-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


***********************************************
Saving GLM for sub-601 ses-02 task-yesNo
***********************************************


In [7]:
from compute_contrasts_utils import compute_contrast

In [ ]:
subnum = '601'
session = '02'
task = 'binaryChoice'
mnum = 'model1'
space = 'MNI152NLin2009cAsym_res-2'
output_type = 'effect_size'
out_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri/derivatives/nilearn/glm/level1/%s/%s'%(task, mnum)
contrasts_fn = 'binaryChoice_model1_contrasts.json'

# compute_contrast(subnum, session, task, mnum, contrasts_fn, out_path, output_type = output_type, space = space)

In [8]:
subnum = '601'
session = '02'
task = 'yesNo'
mnum = 'model1'
space = 'MNI152NLin2009cAsym_res-2'
output_type = 'effect_size'
out_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri/derivatives/nilearn/glm/level1/%s/%s'%(task, mnum)
contrasts_fn = 'yesNo_model1_contrasts.json'

compute_contrast(subnum, session, task, mnum, contrasts_fn, out_path, output_type = output_type, space = space)

***********************************************
Loading GLM for sub-601 ses-02 task-yesNo
***********************************************
***********************************************
Loading contrast file yesNo_model1_contrasts.json
***********************************************
***********************************************
Computing contrasts for sub-601 ses-02 task-yesNo
***********************************************


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-nvr-fmri/lib/python3.8/site-packages/nilearn/glm/first_level/first_level.py:652: UserWarning: One contrast given, assuming it for all 2 runs
  warn('One contrast given, assuming it for all %d runs' % n_runs)


***********************************************
Done saving contrasts for sub-601 ses-02 task-yesNo
***********************************************
